<img height="100" src="pic.jpg" width="900" alt="roey zalta" id="tmg_logo" title="Data Analyst Course"><b><div style ='background:lightgreen;dont-size:15px'><center><h1><b>World Cup 2022 - Poisson Distribution</b></h1></center>


<b><div style ='background:lightgreen;dont-size:15px'><h2>So What is poisson distribution  process? here's a quick explantion </h2></b> 


<img height="1000" src="pic3.png" width="1000" alt="roey zalta" id="tmg_logo" title="Data Analyst Course">

<center><b><div style ='background:lightgreen;dont-size:15px'> <h2>  Import Libaries & Data </h2> </b></center>

In [1]:
#import libaries
import pandas as pd
import pickle
from scipy.stats import poisson

!pip install scipy

In [2]:
#import data
dict_table = pickle.load(open('dict_table','rb'))
df_historical_data = pd.read_csv('clean_fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('clean_fifa_worldcup_fixture.csv')

<center><b><div style ='background:lightgreen;dont-size:15px'> <h2>  calculate Team's Strength </h2> </b></center>

In [3]:
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]

df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


In [4]:
#function predicts points
#the function predicts the probabilty of each goal for each team
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

<center><b><div style ='background:lightgreen;dont-size:15px'> <h2>  Testing Function </h2> </b></center>

In [5]:
print(predict_points('England', 'United States'))
print(predict_points('Argentina', 'Mexico'))
print(predict_points('Qatar (H)', 'Ecuador')) # Qatar vs Team X -> 0 points to both

(2.2356147635326007, 0.5922397535606193)
(2.3129151525530505, 0.5378377125059863)
(0, 0)


<center><b><div style ='background:lightgreen;dont-size:15px'> <h2>  World Cup 2022 - Prediction </h2> </b></center>

In [6]:
#splitting into 48 matches , knockout stage , quarter, semi and finals scores tables
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [7]:
# 
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [20]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout['home points'] = round(points_home,0)
df_fixture_knockout['away points'] = round(points_away,0)
df_fixture_knockout

,home,score,away,year,winner,home points,away points
48,Netherlands,Match 49,Wales,2022,?,0.0,2.0
49,Argentina,Match 50,Denmark,2022,?,0.0,2.0
50,France,Match 52,Poland,2022,?,0.0,2.0
51,England,Match 51,Senegal,2022,?,0.0,2.0
52,Germany,Match 53,Belgium,2022,?,0.0,2.0
53,Brazil,Match 54,Uruguay,2022,?,0.0,2.0
54,Croatia,Match 55,Spain,2022,?,0.0,2.0
55,Portugal,Match 56,Switzerland,2022,?,0.0,2.0


In [42]:
def get_winner(df_fixture_group_48):
    for index, row in df_fixture_group_48.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_group_48.loc[index, 'winner'] = winner
        df_fixture_group_48.loc[index,'home points'] = round(points_home,0)
        df_fixture_group_48.loc[index,'away points'] = round(points_away,0)
    return df_fixture_group_48

In [43]:
get_winner(df_fixture_group_48)

,home,score,away,year,winner,home points,away points
0,Qatar,Match 1,Ecuador,2022,Ecuador,0.0,0.0
1,Senegal,Match 2,Netherlands,2022,Netherlands,1.0,2.0
2,Qatar,Match 18,Senegal,2022,Senegal,0.0,0.0
3,Netherlands,Match 19,Ecuador,2022,Netherlands,2.0,1.0
4,Ecuador,Match 35,Senegal,2022,Senegal,1.0,2.0
5,Netherlands,Match 36,Qatar,2022,Qatar,0.0,0.0
6,England,Match 3,Iran,2022,England,2.0,0.0
7,United States,Match 4,Wales,2022,Wales,1.0,2.0
8,Wales,Match 17,Iran,2022,Wales,2.0,1.0
9,England,Match 20,United States,2022,England,2.0,1.0
